In [7]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [4]:
#settings

In [31]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196'

In [11]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [12]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,jQTE9N9ZKbguudoFXjK93siPMsR0RxJ9iWe2ji9sbRkVxy86,Wpw-SB6IFCCehkR02j8k1fbAO7Pn6f9dxlW-RjICs-Xl_o...,OnW1Dw_JX8vapRJEb-ZVjqdKVknpD2IGd6ZWowLTrkmoEe...,kevyzhou,7,1656398187000,1,jQTE9N9ZKbguudoFXjK93siPMsR0RxJ9iWe2ji9sbRkVxy86,kevyzhou,650,I,246,233,False,False,True,False
1,m-qUxgZJLteV8W5Oqwwn50XfK0wXDtalCaVm9reptusp2JY,HG1ubDWkjeBciVDvB0bzOVDPf1SNPNt6Pk23ctu6fyQihA,WhTl_YwHuw06oZYoBvabD1I06ZBEUcelhXUlBDNnPXSKn-...,milala,3838,1656400235000,403,m-qUxgZJLteV8W5Oqwwn50XfK0wXDtalCaVm9reptusp2JY,milala,566,I,159,128,False,False,True,False
2,ZjryB2uaOap3yknlYg2NAM0KT0lJ2kZGxnji3SG0hwrSBgo,MrWHYI6liJ4DJ07BK8x2oTa9cI6yNbE_9tj1PVwrUVoLoc8,JfpGAp6IYvz9ExTxKZ-R0T-oOATXhtI1KRRN5Y4nb6GkOm...,Liquid Kurumx,3440,1656394547000,114,ZjryB2uaOap3yknlYg2NAM0KT0lJ2kZGxnji3SG0hwrSBgo,Liquid Kurumx,1006,I,97,40,False,False,False,True
3,mn3OHPDTDc5CsloPPGlHOQOq0Vqagk66ReC61-8SuXUxqH0,K4FBeziXvtE_px9iFYbp72rXuEhJO_W21lbvRF5VgM4Spw,502mQ2ZxwRwZDIiwzDF_nEvUBsq8qhU9kG_WayPz4qz9DE...,BC Socks,3552,1656399877000,74,mn3OHPDTDc5CsloPPGlHOQOq0Vqagk66ReC61-8SuXUxqH0,BC Socks,1210,I,147,71,True,False,False,False
4,MPDXCwKEH-yDsQmyVYAGl5lzhjZCjAOGp8CSliPtRmWIT9E,B26caZessjQg8cDRq7iPTNJsoK-gn6J4Yc4nRXltcIAEVQ,8T0ADJVvo3K_K4O--Eq-rTIyQ0Pnb_z-EYjDf2QTy3C95O...,TSM FTX Kiyoon,21,1656302298000,148,MPDXCwKEH-yDsQmyVYAGl5lzhjZCjAOGp8CSliPtRmWIT9E,TSM FTX Kiyoon,835,I,136,87,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,nRppazjRzzu6x6SVAZZw__qqNZzoL8jUmz6oA5PcLCFuYis,v0V8za9lUe9Sal3pWMudPbeLKAV9KJam1IhcUktvoCpwFQ,FjLKIVYYUSrV37F_ZcBfkdPfRtkqXlYgw5u8WWv6WI7MYX...,TyIer Franklin,585,1656399634000,37,nRppazjRzzu6x6SVAZZw__qqNZzoL8jUmz6oA5PcLCFuYis,TyIer Franklin,693,I,94,50,False,False,True,True
81,rseOMGMOy6sMmC_BEUN8Rrzur0bedb1TyOJbKvdxThJtAhE,W3cdKoMwYdbDhIFn_7ZYvdTZmXpcdO6UzedkNu_TDBppjPY,lLtsw7rM-yW8RierLNxTKjpGlfn01KJr4Jdovj-7jqNenj...,Ramblinnn,3591,1656311786000,12,rseOMGMOy6sMmC_BEUN8Rrzur0bedb1TyOJbKvdxThJtAhE,Ramblinnn,731,I,97,54,False,False,False,False
82,qIxHRxoAzqUb-gy4NW_VQd_BpacB48qjlem7BGNCXz8Cmj0,k4yVx5UMYLD2EkUnLkCahkVvybxAKTni9NX2n7P2n4ddSg,4OhcQyoMGiGG4HQWXHn15EYNSojNWYHRx3upp3L_xGniI4...,JimLoveAngie,4568,1656387294000,140,qIxHRxoAzqUb-gy4NW_VQd_BpacB48qjlem7BGNCXz8Cmj0,JimLoveAngie,652,I,109,73,False,False,True,False
83,NI1BS8WAKRRk2qPubjPi3XGdv9xaKjPzJQuAzpie33cm-Fxq,qgL9DKswEAkNWKYHk6B4AO2-tlw4a3PfJdf0t0FLPjKT5n...,P6tuguVlHjl_9KVogt65c9N9Q1Hq34UMLAE2pqnweLIQKM...,OatmealWater,5271,1656387693000,48,NI1BS8WAKRRk2qPubjPi3XGdv9xaKjPzJQuAzpie33cm-Fxq,OatmealWater,560,I,160,113,False,False,True,False


In [21]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [22]:
len(uniq_matches_id)

2728

In [23]:
len(matches_asset)

3174

In [26]:
matches_asset[0]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [32]:
latest_matches = [match for match in matches_asset if LATEST_RELEASE in match['info']['game_version']]

In [34]:
len(latest_matches)

1119